In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from itertools import compress
import traceback
from scipy.signal import lfilter, filtfilt
import altair as alt

from load import *
from model import *
from model_epi import *

In [ ]:
"""
Mie predizioni
"""
# Caricamento dati
if 0:
    keys = ['Italia',\
            'Spain', \
            'Germany', \
            'France', \
            'US', \
            'China'
            ]
if 0:
    keys = ['Italia',\
            'EmiliaRomagna', \
            'Lombardia', \
            'Veneto', \
            #'Parma', \
            #'Modena', \
            ]
if 1:
     keys = ['Italia']

df_cases, df_deaths, df_recovered, X, Y_cases, Y_deaths, Y_recovered, Label_cases, Label_deaths, Label_recovered =  load(keys)


# Previsione casi
days_ago = 0
threshold = 50
X_pred_cases, Y_pred_cases, Fit_cases, Rmse_cases, End_date_cases = run_time_model(X, Y_cases, past=days_ago, horizon=90, threshold=threshold)
X_pred_deaths, Y_pred_deaths, Fit_deaths, Rmse_deaths, End_date_deaths = run_time_model(X, Y_deaths, past=days_ago, horizon=90, threshold=threshold)

if 0:
    plot_timeseries("Casi", True, threshold,\
                    X, Y_cases, X_pred_cases, Y_pred_cases, Label_cases)

    print("Predicted end dates:")
    for i, date in enumerate(End_date_cases):
        print("{}\t\t(as of {},\t rmse={}):\t{}".format(Label_cases[i], daynumber2date(max(X[i])-days_ago), Rmse_cases[i], daynumber2date(date)))


# Previsione morti
if 0:
    plot_timeseries("Morti", True, threshold,\
                    X, Y_deaths, X_pred_deaths, Y_pred_deaths, Label_deaths)

    print("Predicted end dates (deaths):")
    for i, date in enumerate(End_date_deaths):
        print("{}\t\t(as of {},\t rmse={}):\t{}".format(Label_deaths[i], daynumber2date(max(X[i])-days_ago), Rmse_deaths[i], daynumber2date(date)))


if 0:
    plot_timeseries2("Casi", True, threshold,\
                     X, Y_cases, X_pred_cases, Y_pred_cases, Label_cases,\
                     X, Y_deaths, X_pred_deaths, Y_pred_deaths, Label_deaths, scale2=7.7)


# Come varia la previsione nel tempo
if 0:
    run_time_model_timemachine(X, Y_cases, Label_cases, threshold=10)


# Dati attuali
N = 5
b = N*[1.0/N]

if 0:
    plot_crossseries("Crescita contagi vs contagi totali", "Cases", "New cases", \
                     np.log10([Y_cases[i][1:] for i in range(len(Y_cases))]), \
                     np.log10(lfilter(b, [1.], np.diff(Y_cases))), \
                     Label_cases)

if 0:
    plot_crossseries("Crescita morti vs morti totali", "Cases", "New cases", \
                     np.log10([Y_deaths[i][1:] for i in range(len(Y_deaths))]), \
                     np.log10(lfilter(b, [1.], np.diff(Y_deaths))), \
                     Label_deaths)

if 0:
    plot_crossseries("Morti vs casi totali", "Cases", "New cases", \
                     np.log10(Y_cases), \
                     np.log10(lfilter(b, [1.], Y_deaths)), \
                     Label_deaths)

if 0:
    plot_crossseries("Crescita morti vs casi totali", "Cases", "New cases", \
                     np.log10([Y_cases[i][1:] for i in range(len(Y_cases))]), \
                     np.log10(lfilter(b, [1.], np.diff(Y_deaths))), \
                     Label_deaths)



In [2]:
"""
Nuovo lavoro preso da: https://www.kaggle.com/volpatto/covid-19-study-with-epidemiology-models
"""

def altair_plot_for_confirmed_and_deaths(df_grouped: pd.DataFrame):
    
    confirmed_plot = alt.Chart(df_grouped).mark_circle(size=60).encode(
        x=alt.X('date_n', axis=alt.Axis(title='Date')),
        y=alt.Y('confirmed', axis=alt.Axis(title='Cases'), title='Confirmed'),
        color=alt.Color("confirmed_marker", title="Cases"),
    )

    """
    deaths_plot = alt.Chart(df_grouped).mark_circle(size=60).encode(
        x='date_n',
        y='deaths',
        color=alt.Color("deaths_marker"),
    )
    """

    return confirmed_plot


df_cases, df_deaths, df_recovered, X, Y_cases, Y_deaths, Y_recovered, Label_cases, Label_deaths, Label_recovered =  load([])
df_pop = load_population()


"""
Initial conditions
"""
df_target_country = country(df_cases, df_deaths, df_recovered, "Italy")
df_target_country = trim_country(df_target_country)
target_population = get_target_population(df_pop, 'Italy')
print(df_target_country)
altair_plot_for_confirmed_and_deaths(df_target_country)

S0, E0, I0, R0, D0 = target_population, 5 * float(df_target_country.confirmed[0]), float(df_target_country.confirmed[0]), 0., 0.
y0_sir = S0 / target_population, I0 / target_population, R0  # SIR IC array
y0_sird = S0 / target_population, I0 / target_population, R0, D0  # SIRD IC array
y0_seir = S0 / target_population, E0 / target_population, I0 / target_population, R0  # SEIR IC array
y0_seird = S0 / target_population, E0 / target_population, I0 / target_population, R0, D0  # SEIRD IC array

"""
What to run
"""
has_to_run_sir = True
has_to_run_sird = False
has_to_run_seir = True
has_to_run_seird = False
has_to_run_seirdq = True



Elements in df_cases: 87
Elements in df_deaths: 87
Elements in df_recovered: 87
    date_n        date  confirmed   deaths  recovered confirmed_marker  \
0       31  2020-01-31        2.0      0.0        0.0        Confirmed   
1       32  2020-02-01        2.0      0.0        0.0        Confirmed   
2       33  2020-02-02        2.0      0.0        0.0        Confirmed   
3       34  2020-02-03        2.0      0.0        0.0        Confirmed   
4       35  2020-02-04        2.0      0.0        0.0        Confirmed   
..     ...         ...        ...      ...        ...              ...   
73     104  2020-04-13   159516.0  20465.0    35435.0        Confirmed   
74     105  2020-04-14   162488.0  21067.0    37130.0        Confirmed   
75     106  2020-04-15   165155.0  21645.0    38092.0        Confirmed   
76     107  2020-04-16   168941.0  22170.0    40164.0        Confirmed   
77     108  2020-04-17   172434.0  22745.0    42727.0        Confirmed   

   deaths_marker recovered_mark

In [ ]:
x = np.arange(100)
source = pd.DataFrame({
  'x': x,
  'f(x)': np.sin(x / 5)
})

alt.Chart(source).mark_line().encode(
    x='x',
    y='f(x)'
)


